# VRI anomaly detection bites

This notebook has the aim to study how to detect anomalies in the VRI computed by our models displayed here: https://labs.mosquitoalert.com/MosquitoAlertES/

Data gathered from models bites.

## Requirements

In [2]:
import pandas as pd
import geopandas as gpd
from prophet import Prophet
from prophet.plot import seasonality_plot_df
import os
from tqdm import tqdm

Importing plotly failed. Interactive plots will not work.


## Directories and Files

In [3]:
# * Base directory. # TODO: Change this to the cluster directory
DATA_DIR = os.path.join(os.getcwd(), 'data')

# * Input
INPUT_DIR = os.path.join(DATA_DIR, 'input')
# Bites data
BITES_DATA_DIR = os.path.join(INPUT_DIR, 'bites')
# GEO data
GEO_DATA_DIR = os.path.join(INPUT_DIR, 'geo')

# * Output
OUTPUT_DIR = os.path.join(DATA_DIR, 'output')
# Anomaly and seasonality output
ANOMALY_OUTPUT_DIR = os.path.join(OUTPUT_DIR, 'spain_activty_anomaly_bites.csv')
SEASONALITY_OUTPUT_DIR = os.path.join(OUTPUT_DIR, 'spain_seasonality_bites.csv')
GPK_PATH = os.path.join(OUTPUT_DIR, 'output_bites.gpkg')
os.makedirs(OUTPUT_DIR, exist_ok=True)

## Dataset

In [ ]:
# Collect all CSV file paths
files = [
    os.path.join(root, file)
    for root, _, files in os.walk(BITES_DATA_DIR)
    for file in files if file.endswith(".csv")
]

# Initialize an empty list to hold the data
dfs = []
# Loop through the files
for file in files:
    try:
        date = file.split("bites_")[1].split(".")[0]
        df_day = pd.read_csv(file)
        df_day["date"] = date
        dfs.append(df_day)
    except Exception as e:
        print(f"Error processing file {file}: {e}")

# Create a DataFrame from the list of data
df = pd.concat(dfs, ignore_index=True)
del dfs

In [3]:
df

,laucode,est,date
0,4001,0.669,2023-08-31
1,4002,0.662,2023-08-31
2,4003,0.716,2023-08-31
3,4004,0.686,2023-08-31
4,4005,0.715,2023-08-31
...,...,...,...
15819370,26181,0.329,2024-10-20
15819371,26183,0.315,2024-10-20
15819372,53056,0.342,2024-10-20
15819373,51001,0.676,2024-10-20


In [4]:
# Rename columns for Prophet
df['ds'] = pd.to_datetime(df["date"])
df.rename(columns={"est": "y"}, inplace=True)

df.sort_values(by=['laucode', 'ds'], inplace=True, ignore_index=True)

# Keep only values for laucode, ds, y
df = df[['ds', 'laucode', 'y']]

df

,ds,laucode,y
0,2020-01-01,1001,0.153
1,2020-01-02,1001,0.189
2,2020-01-03,1001,0.189
3,2020-01-04,1001,0.189
4,2020-01-05,1001,0.189
...,...,...,...
15819370,2025-04-26,53083,0.189
15819371,2025-04-27,53083,0.189
15819372,2025-04-28,53083,0.189
15819373,2025-04-29,53083,0.189


In [ ]:
import logging
logger = logging.getLogger('cmdstanpy')
logger.addHandler(logging.NullHandler())
logger.propagate = False
logger.setLevel(logging.CRITICAL)

import warnings
warnings.filterwarnings("ignore", category=pd.errors.SettingWithCopyWarning)

# Function to train a model and detect anomalies for each city
def detect_anomalies_for_city(city_data):
    group_name, city_df = city_data
    if (city_df['y'].isna()).all() or (city_df['y'] == 0).all():  # Skip if all original items are zero or NaN
        return None, None

    # The following code of false holidays is optional with the new data
    first_non_zero = city_df[city_df["y"] != 0].iloc[0]
    holidays_df = city_df[(city_df['y']==0) & (city_df['ds'] < first_non_zero['ds'])]['ds'].reset_index()
    holidays_df['holiday'] = 'no-prediction-yet'

    # Step 3: Initialize Prophet with logistic growth
    model = Prophet(growth='logistic', yearly_seasonality=True, weekly_seasonality=False, daily_seasonality=False, holidays=holidays_df[['ds','holiday']])
    city_df.loc[:,'cap'] = 1
    city_df.loc[:,'floor'] = 0
    model.fit(city_df)

    # Make predictions for historical data (no future periods).
    # This means that we are not predicting future values, but rather using the model to predict the historical data.
    future = model.make_future_dataframe(periods=0)
    future['cap'] = 1  # Ensure the future data has the cap
    future['floor'] = 0  # Ensure the future data has the floor
    forecast = model.predict(future)

    forecast['fact'] = city_df['y'].reset_index(drop = True)

    forecast['anomaly'] = 0
    forecast.loc[forecast['fact'] > forecast['yhat_upper'], 'anomaly'] = 1
    forecast.loc[forecast['fact'] < forecast['yhat_lower'], 'anomaly'] = -1

     #anomaly importances
    forecast['importance'] = 0.0
    forecast.loc[forecast['anomaly'] ==1, 'importance'] = \
        (forecast['fact'] - forecast['yhat_upper'])/forecast['fact']
    forecast.loc[forecast['anomaly'] ==-1, 'importance'] = \
        (forecast['yhat_lower'] - forecast['fact'])/forecast['fact']

    # Merge forecast with the original data
    city_df_forecast = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper', 'trend', 'anomaly', 'importance']]
    result_df = city_df[['laucode', 'ds']].merge(city_df_forecast, on='ds', how='left')

    # Seasonality component
    df_w = seasonality_plot_df(m=model, ds=pd.date_range(start='2017-01-01', periods=365))
    seas_df = model.predict_seasonal_components(df_w)
    yearly_df = seas_df['yearly'].reset_index()
    yearly_df.loc[:,'laucode'] = city_df.iloc[0]['laucode']

    return result_df, yearly_df

In [6]:
from concurrent.futures import ProcessPoolExecutor
import os
import math

# Apply the anomaly detection for each city in parallel
with ProcessPoolExecutor(max_workers=math.floor(max(os.cpu_count() * 0.8, 1))) as executor:
    results = list(
        tqdm(
            executor.map(
                detect_anomalies_for_city,
                df.groupby('laucode')
            ),
            total=len(
                df['laucode'].unique()
            )
        )
    )

100%|██████████| 8125/8125 [24:24<00:00,  5.55it/s]  


In [7]:
# Combine the results for all cities
result_df = df.merge(
    pd.concat([arr[0] for arr in results if arr is not None]),
    on=['laucode', 'ds'],
    how='left'
)
# Setting a 0 for the prediction value that hasn't been predicted because was all 0.
result_df[['yhat', 'yhat_lower', 'yhat_upper', 'trend', 'anomaly', 'importance']] = result_df[['yhat', 'yhat_lower', 'yhat_upper', 'trend', 'anomaly', 'importance']].fillna(0)
yearly_seasonality_df = pd.concat([arr[1] for arr in results if arr is not None])

In [ ]:
result_df.to_csv(ANOMALY_OUTPUT_DIR, index=False)
yearly_seasonality_df.to_csv(SEASONALITY_OUTPUT_DIR, index=False)

## Part 2

In [4]:
result_df = pd.read_csv(ANOMALY_OUTPUT_DIR)
yearly_seasonality_df = pd.read_csv(SEASONALITY_OUTPUT_DIR)

In [33]:
result_df

,ds,laucode,y,yhat,yhat_lower,yhat_upper,trend,anomaly,importance
0,2020-01-01,1001,0.153,0.175428,0.121867,0.227070,0.292400,0.0,0.0
1,2020-01-02,1001,0.189,0.174842,0.124983,0.231949,0.292400,0.0,0.0
2,2020-01-03,1001,0.189,0.174227,0.122566,0.226680,0.292401,0.0,0.0
3,2020-01-04,1001,0.189,0.173565,0.122012,0.228036,0.292401,0.0,0.0
4,2020-01-05,1001,0.189,0.172842,0.123762,0.223997,0.292402,0.0,0.0
...,...,...,...,...,...,...,...,...,...
15819370,2025-04-26,53083,0.189,0.213411,0.156533,0.270624,0.311072,0.0,0.0
15819371,2025-04-27,53083,0.189,0.214290,0.155988,0.270703,0.310992,0.0,0.0
15819372,2025-04-28,53083,0.189,0.215089,0.155401,0.270857,0.310912,0.0,0.0
15819373,2025-04-29,53083,0.189,0.215835,0.158873,0.272770,0.310832,0.0,0.0


In [5]:
current_status_df = result_df.sort_values(
    by=['laucode', 'ds']
).groupby('laucode').apply(lambda x: x.iloc[-1])[['y', 'yhat', 'yhat_lower', 'yhat_upper', 'trend', 'anomaly', 'importance', 'ds']]

/tmp/ipykernel_5708/2315594896.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ).groupby('laucode').apply(lambda x: x.iloc[-1])[['y', 'yhat', 'yhat_lower', 'yhat_upper', 'trend', 'anomaly', 'importance', 'ds']]


In [6]:
current_status_df.reset_index(inplace=True)

In [7]:
current_status_df.rename(columns={'ds': 'last_update'}, inplace=True)
current_status_df['laucode'] = current_status_df['laucode'].astype(int)

In [8]:
current_status_df

,laucode,y,yhat,yhat_lower,yhat_upper,trend,anomaly,importance,last_update
0,1001,0.154,0.179256,0.128256,0.229727,0.266973,0.0,0.000000,2025-04-30
1,1002,0.197,0.235072,0.177381,0.292163,0.323646,0.0,0.000000,2025-04-30
2,1003,0.211,0.233407,0.174937,0.291270,0.323197,0.0,0.000000,2025-04-30
3,1004,0.206,0.238862,0.181408,0.295719,0.326545,0.0,0.000000,2025-04-30
4,1006,0.153,0.170787,0.118127,0.222901,0.264013,0.0,0.000000,2025-04-30
...,...,...,...,...,...,...,...,...,...
8120,53077,0.144,0.207336,0.152267,0.262323,0.312064,-1.0,0.057411,2025-04-30
8121,53078,0.225,0.262624,0.210472,0.315507,0.349230,0.0,0.000000,2025-04-30
8122,53080,0.184,0.236635,0.179756,0.293412,0.323638,0.0,0.000000,2025-04-30
8123,53081,0.205,0.231629,0.178065,0.284201,0.322165,0.0,0.000000,2025-04-30


### Load shapefiles & save geopackage

### WITH GADM

In [ ]:
basemap_gdf = gpd.read_file(os.path.join(INPUT_DIR, 'geo', 'GADM', 'gadm_410_esp_clean.gpkg'))
basemap_gdf.head(10)

,uid,continent,gid_0,name_0,varname_0,gid_1,name_1,varname_1,nl_name_1,type_1,...,gid_4,name_4,varname_4,type_4,engtype_4,gid_5,name_5,type_5,engtype_5,geometry
0,299189,Europe,ESP,Spain,,ESP.1_1,Andalucía,Andalousie|Andaluc¡a|Andalusien|,,Comunidad Autónoma,...,ESP.1.1.1.1_1,Albánchez,,Municipality,Municipality,,,,,"MULTIPOLYGON (((-2.20208 37.31227, -2.19354 37..."
1,299190,Europe,ESP,Spain,,ESP.1_1,Andalucía,Andalousie|Andaluc¡a|Andalusien|,,Comunidad Autónoma,...,ESP.1.1.1.2_1,Albox,,Municipality,Municipality,,,,,"MULTIPOLYGON (((-2.05695 37.44172, -2.06369 37..."
2,299191,Europe,ESP,Spain,,ESP.1_1,Andalucía,Andalousie|Andaluc¡a|Andalusien|,,Comunidad Autónoma,...,ESP.1.1.1.3_1,Alcóntar,,Municipality,Municipality,,,,,"MULTIPOLYGON (((-2.60811 37.41111, -2.58768 37..."
3,299192,Europe,ESP,Spain,,ESP.1_1,Andalucía,Andalousie|Andaluc¡a|Andalusien|,,Comunidad Autónoma,...,ESP.1.1.1.4_1,Arboleas,,Municipality,Municipality,,,,,"MULTIPOLYGON (((-2.13104 37.29965, -2.13354 37..."
4,299193,Europe,ESP,Spain,,ESP.1_1,Andalucía,Andalousie|Andaluc¡a|Andalusien|,,Comunidad Autónoma,...,ESP.1.1.1.5_1,Armuña de Almanzora,,Municipality,Municipality,,,,,"MULTIPOLYGON (((-2.4198 37.33429, -2.42154 37...."
5,299194,Europe,ESP,Spain,,ESP.1_1,Andalucía,Andalousie|Andaluc¡a|Andalusien|,,Comunidad Autónoma,...,ESP.1.1.1.6_1,Bacares,,Municipality,Municipality,,,,,"MULTIPOLYGON (((-2.47734 37.28898, -2.4748 37...."
6,299195,Europe,ESP,Spain,,ESP.1_1,Andalucía,Andalousie|Andaluc¡a|Andalusien|,,Comunidad Autónoma,...,ESP.1.1.1.7_1,Bayarque,,Municipality,Municipality,,,,,"MULTIPOLYGON (((-2.42154 37.34183, -2.4198 37...."
7,299196,Europe,ESP,Spain,,ESP.1_1,Andalucía,Andalousie|Andaluc¡a|Andalusien|,,Comunidad Autónoma,...,ESP.1.1.1.8_1,Cantoria,,Municipality,Municipality,,,,,"MULTIPOLYGON (((-2.11904 37.28019, -2.12362 37..."
8,299197,Europe,ESP,Spain,,ESP.1_1,Andalucía,Andalousie|Andaluc¡a|Andalusien|,,Comunidad Autónoma,...,ESP.1.1.1.9_1,Chercos,,Municipality,Municipality,,,,,"MULTIPOLYGON (((-2.27054 37.28481, -2.26191 37..."
9,299198,Europe,ESP,Spain,,ESP.1_1,Andalucía,Andalousie|Andaluc¡a|Andalusien|,,Comunidad Autónoma,...,ESP.1.1.1.10_1,Fines,,Municipality,Municipality,,,,,"MULTIPOLYGON (((-2.25195 37.37335, -2.24889 37..."


### WITH IGN

In [ ]:
peninsula_ccaa_ign = gpd.read_file(os.path.join(GEO_DATA_DIR, 'lineas_limite', 'SHP_ETRS89', 'recintos_autonomicas_inspire_peninbal_etrs89'))
peninsula_ccaa_ign = peninsula_ccaa_ign.to_crs(epsg=4326)

canarias_ccaa_ign = gpd.read_file(os.path.join(GEO_DATA_DIR, 'lineas_limite', 'SHP_REGCAN95', 'recintos_autonomicas_inspire_canarias_regcan95'))
canarias_ccaa_ign = canarias_ccaa_ign.to_crs(epsg=4326)

spain_ccaa_ign = gpd.GeoDataFrame(pd.concat([peninsula_ccaa_ign, canarias_ccaa_ign], ignore_index=True))
spain_ccaa_ign['NAMEUNIT'] = spain_ccaa_ign['NAMEUNIT'].str.split('/').str[0]
spain_ccaa_ign.head(10)

,INSPIREID,COUNTRY,NATLEV,NATLEVNAME,NATCODE,NAMEUNIT,CODNUT1,CODNUT2,CODNUT3,geometry
0,ES.IGN.BDDAE.34010000000,ES,https://inspire.ec.europa.eu/codelist/Administ...,Comunidad autónoma,34010000000,Andalucía,ES6,ES61,None,"MULTIPOLYGON (((-6.3176 36.5288, -6.31759 36.5..."
1,ES.IGN.BDDAE.34020000000,ES,https://inspire.ec.europa.eu/codelist/Administ...,Comunidad autónoma,34020000000,Aragón,ES2,ES24,None,"POLYGON ((-1.68752 40.57939, -1.68582 40.57991..."
2,ES.IGN.BDDAE.34030000000,ES,https://inspire.ec.europa.eu/codelist/Administ...,Comunidad autónoma,34030000000,Principado de Asturias,ES1,ES12,None,"MULTIPOLYGON (((-6.85727 43.17325, -6.85767 43..."
3,ES.IGN.BDDAE.34040000000,ES,https://inspire.ec.europa.eu/codelist/Administ...,Comunidad autónoma,34040000000,Illes Balears,ES5,ES53,None,"MULTIPOLYGON (((1.24355 38.86118, 1.2436 38.86..."
4,ES.IGN.BDDAE.34060000000,ES,https://inspire.ec.europa.eu/codelist/Administ...,Comunidad autónoma,34060000000,Cantabria,ES1,ES13,None,"MULTIPOLYGON (((-4.15851 42.86621, -4.16084 42..."
5,ES.IGN.BDDAE.34070000000,ES,https://inspire.ec.europa.eu/codelist/Administ...,Comunidad autónoma,34070000000,Castilla y León,ES4,ES41,None,"MULTIPOLYGON (((-6.86233 40.27808, -6.86223 40..."
6,ES.IGN.BDDAE.34080000000,ES,https://inspire.ec.europa.eu/codelist/Administ...,Comunidad autónoma,34080000000,Castilla-La Mancha,ES4,ES42,None,"MULTIPOLYGON (((-4.49534 38.44132, -4.49591 38..."
7,ES.IGN.BDDAE.34090000000,ES,https://inspire.ec.europa.eu/codelist/Administ...,Comunidad autónoma,34090000000,Cataluña,ES5,ES51,None,"MULTIPOLYGON (((0.6478 40.57446, 0.64803 40.57..."
8,ES.IGN.BDDAE.34100000000,ES,https://inspire.ec.europa.eu/codelist/Administ...,Comunidad autónoma,34100000000,Comunitat Valenciana,ES5,ES52,None,"MULTIPOLYGON (((-0.75231 37.88643, -0.75235 37..."
9,ES.IGN.BDDAE.34110000000,ES,https://inspire.ec.europa.eu/codelist/Administ...,Comunidad autónoma,34110000000,Extremadura,ES4,ES43,None,"POLYGON ((-6.81314 38.12621, -6.81316 38.12644..."


In [99]:
peninsula_ign = gpd.read_file(os.path.join(GEO_DATA_DIR, 'lineas_limite', 'SHP_ETRS89', 'recintos_municipales_inspire_peninbal_etrs89'))
peninsula_ign = peninsula_ign.to_crs(epsg=4326)

canarias_ign = gpd.read_file(os.path.join(GEO_DATA_DIR, 'lineas_limite', 'SHP_REGCAN95', 'recintos_municipales_inspire_canarias_regcan95'))
canarias_ign = canarias_ign.to_crs(epsg=4326)

spain_ign = gpd.GeoDataFrame(pd.concat([peninsula_ign, canarias_ign], ignore_index=True))
spain_ign['NAMEUNIT'] = spain_ign['NAMEUNIT'].str.split('/').str[0]



In [ ]:
gadm4_gdf = gpd.read_file(os.path.join(GEO_DATA_DIR, 'GADM', 'gadm41_ESP.gpkg'), layer='ADM_ADM_4')
# The geometry conversion from polygon to point is necessary for the merge in the next cell
gadm4_gdf['geometry'] = gadm4_gdf.representative_point()
gadm4_gdf.head(10)

,GID_4,GID_0,COUNTRY,GID_1,NAME_1,GID_2,NAME_2,GID_3,NAME_3,NAME_4,VARNAME_4,TYPE_4,ENGTYPE_4,CC_4,geometry
0,ESP.1.1.1.1_1,ESP,Spain,ESP.1_1,Andalucía,ESP.1.1_1,Almería,ESP.1.1.1_1,n.a. (15),Albánchez,NA,Municipality,Municipality,NA,POINT (-2.15973 37.28498)
1,ESP.1.1.1.2_1,ESP,Spain,ESP.1_1,Andalucía,ESP.1.1_1,Almería,ESP.1.1.1_1,n.a. (15),Albox,NA,Municipality,Municipality,NA,POINT (-2.14321 37.44995)
2,ESP.1.1.1.3_1,ESP,Spain,ESP.1_1,Andalucía,ESP.1.1_1,Almería,ESP.1.1.1_1,n.a. (15),Alcóntar,NA,Municipality,Municipality,NA,POINT (-2.6233 37.31929)
3,ESP.1.1.1.4_1,ESP,Spain,ESP.1_1,Andalucía,ESP.1.1_1,Almería,ESP.1.1.1_1,n.a. (15),Arboleas,NA,Municipality,Municipality,NA,POINT (-2.08448 37.36473)
4,ESP.1.1.1.5_1,ESP,Spain,ESP.1_1,Andalucía,ESP.1.1_1,Almería,ESP.1.1.1_1,n.a. (15),Armuña de Almanzora,NA,Municipality,Municipality,NA,POINT (-2.41459 37.35311)
5,ESP.1.1.1.6_1,ESP,Spain,ESP.1_1,Andalucía,ESP.1.1_1,Almería,ESP.1.1.1_1,n.a. (15),Bacares,NA,Municipality,Municipality,NA,POINT (-2.48359 37.25458)
6,ESP.1.1.1.7_1,ESP,Spain,ESP.1_1,Andalucía,ESP.1.1_1,Almería,ESP.1.1.1_1,n.a. (15),Bayarque,NA,Municipality,Municipality,NA,POINT (-2.44965 37.31312)
7,ESP.1.1.1.8_1,ESP,Spain,ESP.1_1,Andalucía,ESP.1.1_1,Almería,ESP.1.1.1_1,n.a. (15),Cantoria,NA,Municipality,Municipality,NA,POINT (-2.17785 37.34786)
8,ESP.1.1.1.9_1,ESP,Spain,ESP.1_1,Andalucía,ESP.1.1_1,Almería,ESP.1.1.1_1,n.a. (15),Chercos,NA,Municipality,Municipality,NA,POINT (-2.27125 37.2608)
9,ESP.1.1.1.10_1,ESP,Spain,ESP.1_1,Andalucía,ESP.1.1_1,Almería,ESP.1.1.1_1,n.a. (15),Fines,NA,Municipality,Municipality,NA,POINT (-2.26592 37.36401)


In [ ]:
# On the right, we have a point that is going to be merged with the polygon on the left in which it is contained
# This merge will add to the IGN data the GID_4 column from the GADM data
municipalities_gdf = gpd.sjoin(spain_ign, gadm4_gdf, how="left")[[
    'GID_4', 'NATCODE', 'NAMEUNIT', 'CODNUT2', 'geometry'
]]
municipalities_gdf

,GID_4,NATCODE,NAMEUNIT,CODNUT2,geometry
0,ESP.1.1.3.1_1,34010404001,Abla,ES61,"POLYGON ((-2.78452 37.0935, -2.7841 37.09476, ..."
1,ESP.1.1.3.2_1,34010404002,Abrucena,ES61,"POLYGON ((-2.88985 37.09212, -2.88953 37.09249..."
2,ESP.1.1.6.1_1,34010404003,Adra,ES61,"POLYGON ((-3.14019 36.78779, -3.13982 36.78802..."
3,ESP.1.1.1.1_1,34010404004,Albanchez,ES61,"POLYGON ((-2.20218 37.31223, -2.20177 37.31227..."
4,ESP.1.1.3.3_1,34010404005,Alboloduy,ES61,"POLYGON ((-2.71288 37.07817, -2.71128 37.08034..."
...,...,...,...,...,...
8215,ESP.14.1.1.15_1,34053535013,Moya,ES70,"POLYGON ((-15.61865 28.03863, -15.61879 28.039..."
8216,ESP.14.1.1.12_1,34053535014,La Oliva,ES70,"MULTIPOLYGON (((-14.03538 28.62042, -14.03538 ..."
8217,ESP.14.1.1.16_1,34053535015,Pájara,ES70,"MULTIPOLYGON (((-14.51249 28.06793, -14.51266 ..."
8218,ESP.14.1.1.13_1,34053535016,Las Palmas de Gran Canaria,ES70,"MULTIPOLYGON (((-15.50742 28.04825, -15.50746 ..."


In [ ]:
# This merges with the CCAA data is going to add the CCAA name to the corresponding municipalities
# Keeps the geometry from the municipalities
gdf = municipalities_gdf[['GID_4', 'NATCODE', 'NAMEUNIT', 'CODNUT2', 'geometry']].merge(
    spain_ccaa_ign[['NAMEUNIT', 'CODNUT2']].rename(columns={'NAMEUNIT': 'NAMEUNIT_NUT2'}),
    on='CODNUT2',
    how='inner'
)
gdf['NATCODE'] = gdf['NATCODE'].astype(int)
gdf

,GID_4,NATCODE,NAMEUNIT,CODNUT2,geometry,NAMEUNIT_NUT2
0,ESP.1.1.3.1_1,34010404001,Abla,ES61,"POLYGON ((-2.78452 37.0935, -2.7841 37.09476, ...",Andalucía
1,ESP.1.1.3.2_1,34010404002,Abrucena,ES61,"POLYGON ((-2.88985 37.09212, -2.88953 37.09249...",Andalucía
2,ESP.1.1.6.1_1,34010404003,Adra,ES61,"POLYGON ((-3.14019 36.78779, -3.13982 36.78802...",Andalucía
3,ESP.1.1.1.1_1,34010404004,Albanchez,ES61,"POLYGON ((-2.20218 37.31223, -2.20177 37.31227...",Andalucía
4,ESP.1.1.3.3_1,34010404005,Alboloduy,ES61,"POLYGON ((-2.71288 37.07817, -2.71128 37.08034...",Andalucía
...,...,...,...,...,...,...
8335,ESP.14.1.1.15_1,34053535013,Moya,ES70,"POLYGON ((-15.61865 28.03863, -15.61879 28.039...",Canarias
8336,ESP.14.1.1.12_1,34053535014,La Oliva,ES70,"MULTIPOLYGON (((-14.03538 28.62042, -14.03538 ...",Canarias
8337,ESP.14.1.1.16_1,34053535015,Pájara,ES70,"MULTIPOLYGON (((-14.51249 28.06793, -14.51266 ...",Canarias
8338,ESP.14.1.1.13_1,34053535016,Las Palmas de Gran Canaria,ES70,"MULTIPOLYGON (((-15.50742 28.04825, -15.50746 ...",Canarias


In [38]:
current_gdf = gdf.merge(current_status_df.rename(columns={'gid_4': 'GID_4'}), on='GID_4')
current_gdf.set_index('NATCODE', inplace=True)
current_gdf.drop(columns=['GID_4'], inplace=True)

KeyError: 'GID_4'

### WITH LAU / NUTS

In [74]:
lau_gdf = gpd.read_file(os.path.join(GEO_DATA_DIR, 'LAU', 'LAU_RG_01M_2023_4326.shp'))
lau_gdf = lau_gdf[lau_gdf['CNTR_CODE'].eq('ES')].sort_values(by=['LAU_NAME'])
lau_gdf.head(40)
# lau_gdf.count()

,GISCO_ID,CNTR_CODE,LAU_ID,LAU_NAME,POP_2023,POP_DENS_2,AREA_KM2,YEAR,geometry
18357,ES_44001,ES,44001,Ababuj,70.0,NaN,1.287780,2023,"POLYGON ((-0.78475 40.56548, -0.77877 40.56114..."
17785,ES_40001,ES,40001,Abades,862.0,NaN,26.959923,2023,"POLYGON ((-4.31755 40.90156, -4.31616 40.90287..."
18749,ES_48001,ES,48001,Abadiño,7720.0,NaN,214.131887,2023,"POLYGON ((-2.62216 43.17252, -2.6154 43.17813,..."
94973,ES_10001,ES,10001,Abadía,339.0,NaN,7.514405,2023,"POLYGON ((-5.97198 40.24153, -6.00577 40.26196..."
87659,ES_27001,ES,27001,Abadín,2203.0,NaN,11.262943,2023,"POLYGON ((-7.59302 43.41768, -7.59082 43.42206..."
54075,ES_09001,ES,09001,Abajas,28.0,NaN,0.798418,2023,"POLYGON ((-3.60584 42.65913, -3.59445 42.65897..."
73225,ES_20001,ES,20001,Abaltzisketa,326.0,NaN,29.166718,2023,"POLYGON ((-2.12164 43.0404, -2.11502 43.05083,..."
38964,ES_30001,ES,30001,Abanilla,6248.0,NaN,26.519672,2023,"POLYGON ((-1.08433 38.34622, -1.05852 38.34556..."
31568,ES_50001,ES,50001,Abanto,87.0,NaN,1.361220,2023,"POLYGON ((-1.63937 41.16037, -1.63909 41.15239..."
18750,ES_48002,ES,48002,Abanto y Ciérvana-Abanto Zierb,9401.0,NaN,580.313266,2023,"POLYGON ((-3.11367 43.29129, -3.11686 43.29636..."


In [44]:
nuts_gdf = gpd.read_file(os.path.join(GEO_DATA_DIR, 'NUTS', 'NUTS_RG_20M_2024_4326.shp'))
# Keep only code from Spain and NUTS-3 level (province level)
nuts_gdf = nuts_gdf[nuts_gdf["CNTR_CODE"].eq("ES") & nuts_gdf["LEVL_CODE"].eq(3)]
nuts_gdf.head(10)

,NUTS_ID,LEVL_CODE,CNTR_CODE,NAME_LATN,NUTS_NAME,MOUNT_TYPE,URBN_TYPE,COAST_TYPE,geometry
1207,ES111,3,ES,A Coruña,A Coruña,1.0,2.0,1.0,"POLYGON ((-7.69974 43.73511, -7.76474 43.50519..."
1208,ES112,3,ES,Lugo,Lugo,1.0,3.0,1.0,"POLYGON ((-7.03184 43.54447, -7.14643 43.37831..."
1209,ES113,3,ES,Ourense,Ourense,1.0,3.0,NaN,"POLYGON ((-7.07707 42.508, -6.82846 42.45021, ..."
1210,ES114,3,ES,Pontevedra,Pontevedra,1.0,2.0,1.0,"POLYGON ((-8.74396 42.64722, -8.46277 42.74664..."
1211,ES120,3,ES,Asturias,Asturias,1.0,2.0,1.0,"POLYGON ((-4.5123 43.3932, -4.56386 43.29602, ..."
1212,ES130,3,ES,Cantabria,Cantabria,1.0,2.0,1.0,"POLYGON ((-3.15334 43.35322, -3.34442 43.27348..."
1213,ES211,3,ES,Araba/Álava,Araba/Álava,1.0,1.0,NaN,"POLYGON ((-2.25081 42.89569, -2.2957 42.74664,..."
1214,ES212,3,ES,Gipuzkoa,Gipuzkoa,1.0,1.0,1.0,"POLYGON ((-1.78598 43.35058, -1.7289 43.29609,..."
1215,ES213,3,ES,Bizkaia,Bizkaia,1.0,1.0,1.0,"POLYGON ((-2.41285 43.32108, -2.54639 43.08801..."
1216,ES220,3,ES,Navarra,Navarra,1.0,2.0,NaN,"MULTIPOLYGON (((-0.93031 42.98921, -0.7966 42...."


In [45]:
# gadm4_gdf = gpd.list_layers(os.path.join(GEO_DATA_DIR, 'GADM', 'gadm41_ESP.gpkg'))
gadm4_gdf = gpd.read_file(os.path.join(GEO_DATA_DIR, 'GADM', 'gadm41_ESP.gpkg'), layer='ADM_ADM_4')
# gadm4_gdf['geometry'] = gadm4_gdf.representative_point()
gadm4_gdf.head(10)

,GID_4,GID_0,COUNTRY,GID_1,NAME_1,GID_2,NAME_2,GID_3,NAME_3,NAME_4,VARNAME_4,TYPE_4,ENGTYPE_4,CC_4,geometry
0,ESP.1.1.1.1_1,ESP,Spain,ESP.1_1,Andalucía,ESP.1.1_1,Almería,ESP.1.1.1_1,n.a. (15),Albánchez,NA,Municipality,Municipality,NA,"MULTIPOLYGON (((-2.20208 37.31227, -2.19354 37..."
1,ESP.1.1.1.2_1,ESP,Spain,ESP.1_1,Andalucía,ESP.1.1_1,Almería,ESP.1.1.1_1,n.a. (15),Albox,NA,Municipality,Municipality,NA,"MULTIPOLYGON (((-2.05695 37.44172, -2.06369 37..."
2,ESP.1.1.1.3_1,ESP,Spain,ESP.1_1,Andalucía,ESP.1.1_1,Almería,ESP.1.1.1_1,n.a. (15),Alcóntar,NA,Municipality,Municipality,NA,"MULTIPOLYGON (((-2.60811 37.41111, -2.58768 37..."
3,ESP.1.1.1.4_1,ESP,Spain,ESP.1_1,Andalucía,ESP.1.1_1,Almería,ESP.1.1.1_1,n.a. (15),Arboleas,NA,Municipality,Municipality,NA,"MULTIPOLYGON (((-2.13104 37.29965, -2.13354 37..."
4,ESP.1.1.1.5_1,ESP,Spain,ESP.1_1,Andalucía,ESP.1.1_1,Almería,ESP.1.1.1_1,n.a. (15),Armuña de Almanzora,NA,Municipality,Municipality,NA,"MULTIPOLYGON (((-2.4198 37.33429, -2.42154 37...."
5,ESP.1.1.1.6_1,ESP,Spain,ESP.1_1,Andalucía,ESP.1.1_1,Almería,ESP.1.1.1_1,n.a. (15),Bacares,NA,Municipality,Municipality,NA,"MULTIPOLYGON (((-2.47734 37.28898, -2.4748 37...."
6,ESP.1.1.1.7_1,ESP,Spain,ESP.1_1,Andalucía,ESP.1.1_1,Almería,ESP.1.1.1_1,n.a. (15),Bayarque,NA,Municipality,Municipality,NA,"MULTIPOLYGON (((-2.42154 37.34183, -2.4198 37...."
7,ESP.1.1.1.8_1,ESP,Spain,ESP.1_1,Andalucía,ESP.1.1_1,Almería,ESP.1.1.1_1,n.a. (15),Cantoria,NA,Municipality,Municipality,NA,"MULTIPOLYGON (((-2.12362 37.28198, -2.12588 37..."
8,ESP.1.1.1.9_1,ESP,Spain,ESP.1_1,Andalucía,ESP.1.1_1,Almería,ESP.1.1.1_1,n.a. (15),Chercos,NA,Municipality,Municipality,NA,"MULTIPOLYGON (((-2.27054 37.28481, -2.26191 37..."
9,ESP.1.1.1.10_1,ESP,Spain,ESP.1_1,Andalucía,ESP.1.1_1,Almería,ESP.1.1.1_1,n.a. (15),Fines,NA,Municipality,Municipality,NA,"MULTIPOLYGON (((-2.25195 37.37335, -2.24889 37..."


In [12]:
# gadm4_world_gdf = gpd.list_layers(os.path.join(GEO_DATA_DIR, 'GADM', 'gadm_410.gpkg'))
gadm4_world_gdf = gpd.read_file(os.path.join(GEO_DATA_DIR, 'GADM', 'gadm_410.gpkg'))
gadm4_world_gdf.columns

Index(['UID', 'GID_0', 'NAME_0', 'VARNAME_0', 'GID_1', 'NAME_1', 'VARNAME_1',
       'NL_NAME_1', 'ISO_1', 'HASC_1', 'CC_1', 'TYPE_1', 'ENGTYPE_1',
       'VALIDFR_1', 'GID_2', 'NAME_2', 'VARNAME_2', 'NL_NAME_2', 'HASC_2',
       'CC_2', 'TYPE_2', 'ENGTYPE_2', 'VALIDFR_2', 'GID_3', 'NAME_3',
       'VARNAME_3', 'NL_NAME_3', 'HASC_3', 'CC_3', 'TYPE_3', 'ENGTYPE_3',
       'VALIDFR_3', 'GID_4', 'NAME_4', 'VARNAME_4', 'CC_4', 'TYPE_4',
       'ENGTYPE_4', 'VALIDFR_4', 'GID_5', 'NAME_5', 'CC_5', 'TYPE_5',
       'ENGTYPE_5', 'GOVERNEDBY', 'SOVEREIGN', 'DISPUTEDBY', 'REGION',
       'VARREGION', 'COUNTRY', 'CONTINENT', 'SUBCONT', 'geometry'],
      dtype='object')

In [13]:
# gadm4_world_gdf[gadm4_world_gdf["GID_0"].eq("ESP")].head(10)
gadm4_world_gdf[gadm4_world_gdf["GID_0"].eq("ESP")][['UID', 'GID_0', 'NAME_0', 'GID_1', 'NAME_1', 'GID_2', 'NAME_2', 'GID_4', 'NAME_4', 'COUNTRY', 'CONTINENT', 'geometry']].head(10)

,UID,GID_0,NAME_0,GID_1,NAME_1,GID_2,NAME_2,GID_4,NAME_4,COUNTRY,CONTINENT,geometry
299188,299189,ESP,Spain,ESP.1_1,Andalucía,ESP.1.1_1,Almería,ESP.1.1.1.1_1,Albánchez,Spain,Europe,"MULTIPOLYGON (((-2.20208 37.31227, -2.19354 37..."
299189,299190,ESP,Spain,ESP.1_1,Andalucía,ESP.1.1_1,Almería,ESP.1.1.1.2_1,Albox,Spain,Europe,"MULTIPOLYGON (((-2.05695 37.44172, -2.06369 37..."
299190,299191,ESP,Spain,ESP.1_1,Andalucía,ESP.1.1_1,Almería,ESP.1.1.1.3_1,Alcóntar,Spain,Europe,"MULTIPOLYGON (((-2.60811 37.41111, -2.58768 37..."
299191,299192,ESP,Spain,ESP.1_1,Andalucía,ESP.1.1_1,Almería,ESP.1.1.1.4_1,Arboleas,Spain,Europe,"MULTIPOLYGON (((-2.13104 37.29965, -2.13354 37..."
299192,299193,ESP,Spain,ESP.1_1,Andalucía,ESP.1.1_1,Almería,ESP.1.1.1.5_1,Armuña de Almanzora,Spain,Europe,"MULTIPOLYGON (((-2.4198 37.33429, -2.42154 37...."
299193,299194,ESP,Spain,ESP.1_1,Andalucía,ESP.1.1_1,Almería,ESP.1.1.1.6_1,Bacares,Spain,Europe,"MULTIPOLYGON (((-2.47734 37.28898, -2.4748 37...."
299194,299195,ESP,Spain,ESP.1_1,Andalucía,ESP.1.1_1,Almería,ESP.1.1.1.7_1,Bayarque,Spain,Europe,"MULTIPOLYGON (((-2.42154 37.34183, -2.4198 37...."
299195,299196,ESP,Spain,ESP.1_1,Andalucía,ESP.1.1_1,Almería,ESP.1.1.1.8_1,Cantoria,Spain,Europe,"MULTIPOLYGON (((-2.11904 37.28019, -2.12362 37..."
299196,299197,ESP,Spain,ESP.1_1,Andalucía,ESP.1.1_1,Almería,ESP.1.1.1.9_1,Chercos,Spain,Europe,"MULTIPOLYGON (((-2.27054 37.28481, -2.26191 37..."
299197,299198,ESP,Spain,ESP.1_1,Andalucía,ESP.1.1_1,Almería,ESP.1.1.1.10_1,Fines,Spain,Europe,"MULTIPOLYGON (((-2.25195 37.37335, -2.24889 37..."


In [46]:
# The geometry conversion from polygon to point is necessary for the merge in the next cell
nuts_gdf['geometry'] = nuts_gdf.representative_point()
nuts_gdf.head(10)

,NUTS_ID,LEVL_CODE,CNTR_CODE,NAME_LATN,NUTS_NAME,MOUNT_TYPE,URBN_TYPE,COAST_TYPE,geometry
1207,ES111,3,ES,A Coruña,A Coruña,1.0,2.0,1.0,POINT (-8.49044 43.18627)
1208,ES112,3,ES,Lugo,Lugo,1.0,3.0,1.0,POINT (-7.42547 43.05996)
1209,ES113,3,ES,Ourense,Ourense,1.0,3.0,NaN,POINT (-7.53871 42.20401)
1210,ES114,3,ES,Pontevedra,Pontevedra,1.0,2.0,1.0,POINT (-8.52835 42.35128)
1211,ES120,3,ES,Asturias,Asturias,1.0,2.0,1.0,POINT (-5.84571 43.25809)
1212,ES130,3,ES,Cantabria,Cantabria,1.0,2.0,1.0,POINT (-4.25252 43.12497)
1213,ES211,3,ES,Araba/Álava,Araba/Álava,1.0,1.0,NaN,POINT (-2.70906 42.83253)
1214,ES212,3,ES,Gipuzkoa,Gipuzkoa,1.0,1.0,1.0,POINT (-2.24931 43.1123)
1215,ES213,3,ES,Bizkaia,Bizkaia,1.0,1.0,1.0,POINT (-2.94789 43.21445)
1216,ES220,3,ES,Navarra,Navarra,1.0,2.0,NaN,POINT (-1.81487 42.5535)


In [47]:
municipalities_gdf = gpd.sjoin(lau_gdf, nuts_gdf, how="left")[[
    'CNTR_CODE_left', 'LAU_ID', 'LAU_NAME', 'NUTS_NAME', 'geometry'
]].rename(columns={
    'CNTR_CODE_left': 'cntr_code',
    'LAU_ID': 'laucode',
    'LAU_NAME': 'municipality_name',
    'NUTS_NAME': 'province_name'
})
municipalities_gdf['laucode'] = municipalities_gdf['laucode'].astype(int)
municipalities_gdf

,cntr_code,laucode,municipality_name,province_name,geometry
3102,ES,2077,Villa de Ves,NaN,"POLYGON ((-1.22696 39.15172, -1.23754 39.15253..."
3103,ES,2079,Villamalea,NaN,"POLYGON ((-1.56042 39.39896, -1.55565 39.40187..."
3104,ES,2080,Villapalacios,NaN,"POLYGON ((-2.59045 38.59924, -2.57025 38.5882,..."
3105,ES,2081,Villarrobledo,NaN,"POLYGON ((-2.38088 39.15531, -2.40929 39.16669..."
3106,ES,2082,Villatoya,NaN,"POLYGON ((-1.362 39.34948, -1.35282 39.3407, -..."
...,...,...,...,...,...
98358,ES,12028,Benicàssim,NaN,"POLYGON ((0.00268 40.07034, 0.01253 40.07664, ..."
98364,ES,12049,Costur,NaN,"POLYGON ((-0.18575 40.14072, -0.17314 40.14526..."
98380,ES,6017,Baterno,NaN,"POLYGON ((-4.84306 38.94514, -4.84293 38.94082..."
98386,ES,6038,Cordobilla de Lácara,NaN,"POLYGON ((-6.39396 39.13966, -6.39951 39.13579..."


In [48]:
current_gdf = municipalities_gdf.merge(current_status_df, on='laucode')
current_gdf.set_index('laucode', inplace=True)
# current_gdf.drop(columns=['GID_4'], inplace=True)
current_gdf

,cntr_code,municipality_name,province_name,geometry,y,yhat,yhat_lower,yhat_upper,trend,anomaly,importance,last_update
laucode,,,,,,,,,,,,
2077,ES,Villa de Ves,NaN,"POLYGON ((-1.22696 39.15172, -1.23754 39.15253...",0.200,0.260753,0.201065,0.322108,0.366789,-1.0,0.005324,2025-04-30
2079,ES,Villamalea,NaN,"POLYGON ((-1.56042 39.39896, -1.55565 39.40187...",0.179,0.243909,0.187033,0.301807,0.349638,-1.0,0.044876,2025-04-30
2080,ES,Villapalacios,NaN,"POLYGON ((-2.59045 38.59924, -2.57025 38.5882,...",0.188,0.242518,0.185927,0.300125,0.348374,0.0,0.000000,2025-04-30
2081,ES,Villarrobledo,NaN,"POLYGON ((-2.38088 39.15531, -2.40929 39.16669...",0.177,0.242685,0.189205,0.299423,0.344508,-1.0,0.068953,2025-04-30
2082,ES,Villatoya,NaN,"POLYGON ((-1.362 39.34948, -1.35282 39.3407, -...",0.230,0.299149,0.240093,0.361803,0.398673,-1.0,0.043882,2025-04-30
...,...,...,...,...,...,...,...,...,...,...,...,...
12028,ES,Benicàssim,NaN,"POLYGON ((0.00268 40.07034, 0.01253 40.07664, ...",0.380,0.432422,0.374525,0.483231,0.520861,0.0,0.000000,2025-04-30
12049,ES,Costur,NaN,"POLYGON ((-0.18575 40.14072, -0.17314 40.14526...",0.318,0.351412,0.298306,0.409263,0.442017,0.0,0.000000,2025-04-30
6017,ES,Baterno,NaN,"POLYGON ((-4.84306 38.94514, -4.84293 38.94082...",0.261,0.310478,0.255577,0.363265,0.404611,0.0,0.000000,2025-04-30


In [ ]:
historic_gdf = gpd.GeoDataFrame(
    result_df.merge(
        municipalities_gdf[['laucode']],
        on='laucode',
        how='inner'
    ),
    geometry=None
)
historic_gdf

,ds,laucode,y,yhat,yhat_lower,yhat_upper,trend,anomaly,importance
0,2020-01-01,1001,0.153,0.175428,0.121867,0.227070,0.292400,0.0,0.0
1,2020-01-02,1001,0.189,0.174842,0.124983,0.231949,0.292400,0.0,0.0
2,2020-01-03,1001,0.189,0.174227,0.122566,0.226680,0.292401,0.0,0.0
3,2020-01-04,1001,0.189,0.173565,0.122012,0.228036,0.292401,0.0,0.0
4,2020-01-05,1001,0.189,0.172842,0.123762,0.223997,0.292402,0.0,0.0
...,...,...,...,...,...,...,...,...,...
15659716,2025-04-26,52001,0.478,0.488250,0.452114,0.522918,0.568387,0.0,0.0
15659717,2025-04-27,52001,0.478,0.489266,0.454424,0.523558,0.568362,0.0,0.0
15659718,2025-04-28,52001,0.478,0.490252,0.456466,0.526738,0.568337,0.0,0.0
15659719,2025-04-29,52001,0.478,0.491227,0.456447,0.526013,0.568313,0.0,0.0


In [51]:
# Save the GeoPandas DataFrame (geometries)
current_gdf.to_file(GPK_PATH, layer='geometries', driver="GPKG")
historic_gdf.to_file(GPK_PATH, layer='histories', driver="GPKG")